## Web Scraping - Ecommerce de vehículos usados

El mercado de vehīculos usados ha crecido mucho en Perú a raiz de la pandemia ocasionada por el Covid-19. Muchas personas se vieron obligadas a contar con un vehīculo propio para respetar el distanciamiento social, sumado a ello los ingresos extraordinarios que muchas familias recibieron provenientes de la liberación de la CTS y AFP.

Dada la coyuntura, es interesante conocer cual es el tipo de vehículos que se comercializan, a que precios y en que estado. Para ello este proyecto tiene como objetivo obtener información de uno de los ecommerce ya posicionados en el païs, Neoauto.

<img src='/Users/bryanvimar/Documents/Proyectos Python/Scrapping/Curso_Platzi/neoauto.jpg' width="150" height="130">

En este proyecto utilizaremos Python para aplicar técnicas de web scraping y obtener información acerca de vehículos usados, y con este input  elaborar un modelo que nos permita predecir precios de mercado.

### 1. Importar librerías

In [1]:
# Importamos las librerias
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd

### 2. Obtenemos lista total de marcas

In [2]:
url_total_marcas = 'https://neoauto.com/venta-de-autos-usados'
neo = requests.get(url_total_marcas)
s = BeautifulSoup(neo.text, 'html.parser')

In [3]:
#Obtengo todos los item que figuran como filtros de la web, dentro de las cuales están los nombres de las MARCAS.
marcas = s.find_all('a', attrs = {'class', 'c-filter-option__item-row-text'})
lista_marcas = [marca.text for marca in marcas]
lista_marcas

['Toyota',
 'Hyundai',
 'Nissan',
 'Kia',
 'Mazda',
 'Volkswagen',
 'Bmw',
 'Subaru',
 'Honda',
 'Mitsubishi',
 'Acura',
 'Alfa-romeo',
 'Asia',
 'Aston martin',
 'Audi',
 'Baic',
 'Baic yinxiang',
 'Bentley',
 'Brilliance',
 'Byd',
 'Changan',
 'Changhe',
 'Chery',
 'Chevrolet',
 'Chrysler',
 'Citroen',
 'Dacia',
 'Daewoo',
 'Daihatsu',
 'Datsun',
 'Dfsk',
 'Dodge',
 'Dongfeng',
 'Faw',
 'Fiat',
 'Ford',
 'Foton',
 'Geely',
 'Great wall',
 'Haima',
 'Haval',
 'Hino',
 'Hummer',
 'Incapower',
 'Infiniti',
 'International',
 'Isuzu',
 'Jac',
 'Jaguar',
 'Jeep',
 'Jetour',
 'Jinbei',
 'Jmc',
 'Jonway',
 'Keyton',
 'King long',
 'Ktm',
 'Kyc',
 'Lada',
 'Lamborghini',
 'Land rover',
 'Lexus',
 'Lifan',
 'Mahindra',
 'Maserati',
 'Maxus',
 'Mc laren',
 'Mercedes benz',
 'Mg',
 'Mini',
 'Morris',
 'Morris garages',
 'Oldsmobile',
 'Opel',
 'Otros',
 'Peugeot',
 'Polaris',
 'Pontiac',
 'Porsche',
 'Ram',
 'Renault',
 'Scion',
 'Seat',
 'Shineray',
 'Skoda',
 'Smart',
 'Soueast',
 'Ssangyong'

In [4]:
# Filtro la lista y convierto los nombres a minúsculas dado que en las URL no existen mayusculas
lista_marcas_final = []
for i in range(1,93):
    lista_marcas_final.append(lista_marcas[i].lower())

In [5]:
# Hemos obtenido 92 nombres de MARCAS
len(lista_marcas_final)

92

In [6]:
# Dado que en una URL para los nombres compuestos los espacios son reemplazados por guiones, debemos replicar esta transformación en nuestra lista
lista_marcas_oficial = []
for i in range(len(lista_marcas_final)):
    link = lista_marcas_final[i].replace(" ","-")
    lista_marcas_oficial.append(link)

In [7]:
# Visualizo mi lista de marcas final
lista_marcas_oficial

['hyundai',
 'nissan',
 'kia',
 'mazda',
 'volkswagen',
 'bmw',
 'subaru',
 'honda',
 'mitsubishi',
 'acura',
 'alfa-romeo',
 'asia',
 'aston-martin',
 'audi',
 'baic',
 'baic-yinxiang',
 'bentley',
 'brilliance',
 'byd',
 'changan',
 'changhe',
 'chery',
 'chevrolet',
 'chrysler',
 'citroen',
 'dacia',
 'daewoo',
 'daihatsu',
 'datsun',
 'dfsk',
 'dodge',
 'dongfeng',
 'faw',
 'fiat',
 'ford',
 'foton',
 'geely',
 'great-wall',
 'haima',
 'haval',
 'hino',
 'hummer',
 'incapower',
 'infiniti',
 'international',
 'isuzu',
 'jac',
 'jaguar',
 'jeep',
 'jetour',
 'jinbei',
 'jmc',
 'jonway',
 'keyton',
 'king-long',
 'ktm',
 'kyc',
 'lada',
 'lamborghini',
 'land-rover',
 'lexus',
 'lifan',
 'mahindra',
 'maserati',
 'maxus',
 'mc-laren',
 'mercedes-benz',
 'mg',
 'mini',
 'morris',
 'morris-garages',
 'oldsmobile',
 'opel',
 'otros',
 'peugeot',
 'polaris',
 'pontiac',
 'porsche',
 'ram',
 'renault',
 'scion',
 'seat',
 'shineray',
 'skoda',
 'smart',
 'soueast',
 'ssangyong',
 'suzuki'

In [8]:
# Puedo visualizar las marcas en una tabla
df_marcas = pd.DataFrame(lista_marcas_oficial)
df_marcas.head()

,0
0,hyundai
1,nissan
2,kia
3,mazda
4,volkswagen


### 3. Defino la URL

La estructura de la URL que necesitamos tiene 2 input, el nombre de la MARCA y el número de la pagina:

https://neoauto.com/venta-de-autos-usados?busqueda=kia&page=1

Para ello vamos a dividirla para crear una URL por cada MARCA, y luego a cada una de ellas le añadiremos la cantidad de páginas disponibles por cada una de esas búsquedas

### 3.1 Creamos las URL por cada MARCA

In [9]:
url_1 = 'https://neoauto.com/venta-de-autos-usados?busqueda='

In [10]:
url_2 = '&page='

In [11]:
# Creamos un bucle para crear cada una de las URL

urls_marcas = []

for i in lista_marcas_oficial:
    url_marca = url_1 + str(i) + url_2
    urls_marcas.append(url_marca)
print(urls_marcas)

['https://neoauto.com/venta-de-autos-usados?busqueda=hyundai&page=', 'https://neoauto.com/venta-de-autos-usados?busqueda=nissan&page=', 'https://neoauto.com/venta-de-autos-usados?busqueda=kia&page=', 'https://neoauto.com/venta-de-autos-usados?busqueda=mazda&page=', 'https://neoauto.com/venta-de-autos-usados?busqueda=volkswagen&page=', 'https://neoauto.com/venta-de-autos-usados?busqueda=bmw&page=', 'https://neoauto.com/venta-de-autos-usados?busqueda=subaru&page=', 'https://neoauto.com/venta-de-autos-usados?busqueda=honda&page=', 'https://neoauto.com/venta-de-autos-usados?busqueda=mitsubishi&page=', 'https://neoauto.com/venta-de-autos-usados?busqueda=acura&page=', 'https://neoauto.com/venta-de-autos-usados?busqueda=alfa-romeo&page=', 'https://neoauto.com/venta-de-autos-usados?busqueda=asia&page=', 'https://neoauto.com/venta-de-autos-usados?busqueda=aston-martin&page=', 'https://neoauto.com/venta-de-autos-usados?busqueda=audi&page=', 'https://neoauto.com/venta-de-autos-usados?busqueda=bai

### 3.2 Creamos las URL de cada MARCA añadiendo el número de páginas disponible

Tenemos 2 listas, primero los nombres de las MARCAS de autos y luego el número de páginas a buscar

In [12]:
# Creamos una función para obtener las URL con cada una de las páginas disponibles
def contador_paginas(url):

    output_urls = []

    try:
        #Defino maximo numero de paginas por marca
        pages = range(100)
        for i in pages:
            url_f = str(url)
            neo = requests.get(url_f)
            s = BeautifulSoup(neo.text, 'html.parser')
            url_final = url_f + str(i+1)
            last_page = s.find('a', attrs ={'class','c-pagination-content__last-page'}).get('href')
            if url_final == last_page:
                break
            output_urls.append(url_final)
            print(f'Obtención satisfactoria de pagina disponible: {url_final}')

    except Exception as e:
        print(f'Error al intentar obtener la url: {url}')

    return output_urls

In [16]:
# Ahora creamos un bucle para recorrer cada una de las URL de las MARCAS y obtener las URL incluyendo el número de sus páginas disponibles
links_final = []
for i in urls_marcas:
    links_final.append(contador_paginas(i))

Obtención satisfactoria de pagina disponible: https://neoauto.com/venta-de-autos-usados?busqueda=hyundai&page=1
Obtención satisfactoria de pagina disponible: https://neoauto.com/venta-de-autos-usados?busqueda=hyundai&page=2
Obtención satisfactoria de pagina disponible: https://neoauto.com/venta-de-autos-usados?busqueda=hyundai&page=3
Obtención satisfactoria de pagina disponible: https://neoauto.com/venta-de-autos-usados?busqueda=hyundai&page=4
Obtención satisfactoria de pagina disponible: https://neoauto.com/venta-de-autos-usados?busqueda=hyundai&page=5
Obtención satisfactoria de pagina disponible: https://neoauto.com/venta-de-autos-usados?busqueda=hyundai&page=6
Obtención satisfactoria de pagina disponible: https://neoauto.com/venta-de-autos-usados?busqueda=hyundai&page=7
Obtención satisfactoria de pagina disponible: https://neoauto.com/venta-de-autos-usados?busqueda=hyundai&page=8
Obtención satisfactoria de pagina disponible: https://neoauto.com/venta-de-autos-usados?busqueda=hyundai

In [17]:
# Hemos obtenido 92 juegos de URL's de cada una de las marcas.
len(links_final)

92

In [18]:
# Vamos a unir todas las URL disponibles
lista_final = []
for i in range(len(links_final)):
    lista = links_final[i]
    for j in lista:
        lista_final.append(j)

print(lista_final)

['https://neoauto.com/venta-de-autos-usados?busqueda=hyundai&page=1', 'https://neoauto.com/venta-de-autos-usados?busqueda=hyundai&page=2', 'https://neoauto.com/venta-de-autos-usados?busqueda=hyundai&page=3', 'https://neoauto.com/venta-de-autos-usados?busqueda=hyundai&page=4', 'https://neoauto.com/venta-de-autos-usados?busqueda=hyundai&page=5', 'https://neoauto.com/venta-de-autos-usados?busqueda=hyundai&page=6', 'https://neoauto.com/venta-de-autos-usados?busqueda=hyundai&page=7', 'https://neoauto.com/venta-de-autos-usados?busqueda=hyundai&page=8', 'https://neoauto.com/venta-de-autos-usados?busqueda=hyundai&page=9', 'https://neoauto.com/venta-de-autos-usados?busqueda=hyundai&page=10', 'https://neoauto.com/venta-de-autos-usados?busqueda=hyundai&page=11', 'https://neoauto.com/venta-de-autos-usados?busqueda=hyundai&page=12', 'https://neoauto.com/venta-de-autos-usados?busqueda=hyundai&page=13', 'https://neoauto.com/venta-de-autos-usados?busqueda=hyundai&page=14', 'https://neoauto.com/venta-d

In [20]:
# Tenemos 305 páginas disponbiles
len(lista_final)

305

### 4. Definir funciones para enlistar todos los avisos

In [21]:
# Función para enlistar todos los avisos de un link
def enlistar_link_avisos(url_input):

    links_avisos_total = []

    try:
        link = requests.get(url_input)
        if link.status_code == 200:
            s = BeautifulSoup(link.text, 'html.parser')
            # Busco todos los link de los avisos de una pagina
            avisos_page = s.find_all('a', attrs ={'class', 'c-results-use__link'})
            lista_avisos = [aviso.get('href') for aviso in avisos_page]

            for i in range(len(lista_avisos)):
                link = 'https://neoauto.com/'+lista_avisos[i]
                links_avisos_total.append(link)
        else:
            None
    except Exception as e:
        print(f'Error al intentar obtener avisos de la url: {url}')

    return links_avisos_total

### 4.1. Uso función para obtener todos los link de los avisos de cada una de las páginas disponibles de cada MARCA

In [22]:
# Ahora vamos a obtener cada uno de los link de los avisos de los vehículos en venta para todas las páginas que hay de cada MARCA
lista_total_avisos = []
for i in lista_final:
    lista_total_avisos.append(enlistar_link_avisos(i))

In [23]:
# Hemos obtenido todos los link de los avisos. Los cuales están organizados en 315 listas.
len(lista_total_avisos)

305

In [24]:
# Debemos unirlas todos link de los avisos
avisos_disponibles = []
for i in range(len(lista_total_avisos)):
    lista = lista_total_avisos[i]
    for j in lista:
        avisos_disponibles.append(j)

In [25]:
# Hemos obtenido 6,290 avisos.
len(avisos_disponibles)

6100

Hemos identificado que tenemos 6,290 vehīculos que a la fecha están a la venta a través de Neoauto.

### 4. Definir una función para obtener los datos de cada aviso

In [26]:
 # Obtener info de cada aviso
 
def obtener_info(aviso):
    try:

        # Creo un diccionario vacío
        ret_dict = {}

        #Extrar titulo
        titulo = aviso.find('h1', attrs={'class','sc-813e4f1-0 sc-e3e1d7ee-0 fstToR wmLJB'})
        if titulo:
            ret_dict['titulo'] = titulo.text
            
        #Extrar precio
        precio = aviso.find('p', attrs={'class','sc-813e4f1-0 sc-c1459997-0 fstToR dYanzN'})
        if precio:
            ret_dict['precio'] = precio.text

        #Extrar distrito
        distrito = aviso.find('span', attrs={'class','sc-iMJOuO bxjvlv'})
        if distrito:
            ret_dict['distrito'] = distrito.text

        # Extraer caracteristicas varias
        caracteristicas = aviso.find_all('div', attrs={'class','sc-11692204-3 htOtEa'})

        if caracteristicas:
            nombre_datos = []
            for i in range(len(caracteristicas)):
                nombre = caracteristicas[i].text
                nombre_datos.append(nombre)
            ret_dict['caracteristicas'] = ','.join(nombre_datos)
        
        #Extraer Descripción
        descripcion = aviso.find('pre', attrs={'class','sc-c8f2aade-1 hJykuS'})
        if descripcion:
            ret_dict['descripcion'] = descripcion.text

        # Extraer Caracteristicas2
        caracteristicas2 = aviso.find_all('div', attrs={'class','sc-25b7b222-2 jhOymW'})
        
        if caracteristicas2:
            nombre_datos2 = []
            for i in range(len(caracteristicas2)):
                nombre2 = caracteristicas2[i].text
                nombre_datos2.append(nombre2)
            ret_dict['caracteristicas2'] = ','.join(nombre_datos2)

        # Extraer equipamiento
        equipamiento = aviso.find_all('span', attrs={'class','sc-94ecae8-4 ewgDqi'})
        if equipamiento:
            nombre_datos3 = []
            for i in range(len(equipamiento)):
                nombre3 = equipamiento[i].text
                nombre_datos3.append(nombre3)
            ret_dict['equipamiento'] = ','.join(nombre_datos3)

    except Exception as e:
        print('Hubo un error al intentar obtener algún dato')

    return ret_dict

### 5. Crear una función para recrear la acción de scrapear cada aviso

In [27]:
# Función para scrapear cada aviso

def scrape_aviso(url):
    try:
        url = url
        aviso = requests.get(url)
    except Exception as e:
        print('Error scrappeando aviso: ', url)
        print(e)
        return None
    
    if aviso.status_code != 200:
        print(f'Error obteniendo aviso: {url}')
        print(f'Status Code = {aviso.status_code}')
        return None

    l_aviso = BeautifulSoup(aviso.text, 'html.parser')

    ret_dict = obtener_info(l_aviso)
    ret_dict['url'] = url

    return ret_dict

### 6. Procedemos a scrapear toda la información de los avisos disponibles

In [28]:
# Hacemos la prueba con uno de ellos
a = avisos_disponibles[3]

In [29]:
# Vemos el link seleccionado
a

'https://neoauto.com/auto/usado/hyundai-tucson-2017-1685643'

In [30]:
# Usamos nuestra función para obtener la información
scrape_aviso(a)

{'titulo': 'Hyundai Tucson 2017',
 'precio': 'US$ 21,000',
 'distrito': 'Santiago de Surco, Lima, Lima',
 'caracteristicas': '2017,42,300 km,Mecánica,Gasolina,2,000 cc,Impar,Camionetas Suv',
 'descripcion': 'Jr.Cristóbal de Peralta Nte 968, Surco\r\n\r\nAutomotores Gildemeister en su división de Seminuevos ofrece esta unidad . Adquiere este vehículo con toda la seguridad y el respaldo que te brinda el Grupo Gildemeister en nuestra tienda ubicada en Jr.Cristó...Ver más',
 'caracteristicas2': 'Hyundai,TUCSON,2017,5,Gris Oscuro,4',
 'url': 'https://neoauto.com/auto/usado/hyundai-tucson-2017-1685643'}

In [32]:
# Scrapeamos todos los avisos de la pagina
data = []
for i, aviso in enumerate(avisos_disponibles):
    print(f'Scrapeando nota{i}/{len(avisos_disponibles)}')
    data.append(scrape_aviso(aviso))

Scrapeando nota0/6100
Scrapeando nota1/6100
Scrapeando nota2/6100
Scrapeando nota3/6100
Scrapeando nota4/6100
Scrapeando nota5/6100
Scrapeando nota6/6100
Scrapeando nota7/6100
Scrapeando nota8/6100
Scrapeando nota9/6100
Scrapeando nota10/6100
Scrapeando nota11/6100
Scrapeando nota12/6100
Scrapeando nota13/6100
Scrapeando nota14/6100
Scrapeando nota15/6100
Scrapeando nota16/6100
Scrapeando nota17/6100
Scrapeando nota18/6100
Scrapeando nota19/6100
Scrapeando nota20/6100
Scrapeando nota21/6100
Scrapeando nota22/6100
Scrapeando nota23/6100
Scrapeando nota24/6100
Scrapeando nota25/6100
Scrapeando nota26/6100
Scrapeando nota27/6100
Scrapeando nota28/6100
Scrapeando nota29/6100
Scrapeando nota30/6100
Scrapeando nota31/6100
Scrapeando nota32/6100
Scrapeando nota33/6100
Scrapeando nota34/6100
Scrapeando nota35/6100
Scrapeando nota36/6100
Scrapeando nota37/6100
Scrapeando nota38/6100
Scrapeando nota39/6100
Scrapeando nota40/6100
Scrapeando nota41/6100
Scrapeando nota42/6100
Scrapeando nota43/610

### 7. Le damos forma a la información que hemos obtenido

In [33]:
df = pd.DataFrame(data)
df.head(20)

,titulo,precio,distrito,caracteristicas,descripcion,caracteristicas2,equipamiento,url
0,Hyundai Hd35 Euro Iii 2019,"US$ 33,900","La Victoria, Lima, Lima","2019,28,784 km,Mecánica,Diesel,2,500 cc,Camion",*** HYUNDAI HD-35 *** - Versión: TDI CRDI EURO...,"Hyundai,HD35 EURO III,2019,5,4x2,Blanco,4,3,128","Neblineros,Radio CD,Radio MP3,Computadora de a...",https://neoauto.com/bus-camion/usado/hyundai-h...
1,Hyundai I10 2017,"US$ 8,990","San Isidro, Lima, Lima","2017,51,974 km,Mecánica,Gas GNV,1,000 cc,Par,H...","Lunas delanteras electricas, asientos forrados...","Hyundai,i10,2016,5,4x2,Gris Oscuro,3","Neblineros,Radio CD,Aros,Faros antiniebla dela...",https://neoauto.com/auto/usado/hyundai-i10-201...
2,Hyundai Elantra 2020,"US$ 17,400","Lima, Lima, Lima","2020,32,274 km,Automática,Gas GNV,Par,Sedan",Elantra - Ingresa tus datos y te contactaremos...,"Hyundai,ELANTRA,2020,4,4x2,Gris Metálico","Parlantes/Bajos,Retrovisores Eléctricos,Radio ...",https://neoauto.com/auto/usado/hyundai-elantra...
3,Hyundai Tucson 2017,"US$ 21,000","Santiago de Surco, Lima, Lima","2017,42,300 km,Mecánica,Gasolina,2,000 cc,Impa...","Jr.Cristóbal de Peralta Nte 968, Surco\r\n\r\n...","Hyundai,TUCSON,2017,5,Gris Oscuro,4",NaN,https://neoauto.com/auto/usado/hyundai-tucson-...
4,Hyundai Staria 2022,"US$ 34,000","Santiago de Surco, Lima, Lima","2022,80 km,Mecánica,Diesel,2,200 cc,Par,Vans","Jr.Cristóbal de Peralta Nte 968, Surco\r\n\r\n...","Hyundai,STARIA,2022,5,Azul,4",NaN,https://neoauto.com/auto/usado/hyundai-staria-...
5,Hyundai Accent Hb 2013,"US$ 11,500","Surquillo, Lima, Lima","2013,Automática - Secuencial,Gasolina,1,400 cc...","AUTO EN EXCELENTE ESTADO , USO DAMA , PAPELES ...","Hyundai,ACCENT HB,2012,5,Delantera,4","Radio MP3,Retrovisores Eléctricos,Neblineros,A...",https://neoauto.com/auto/usado/hyundai-accent-...
6,Hyundai Tucson 2019,"US$ 24,200","Lima, Lima, Lima","2019,29,579 km,Automática,Gasolina,Impar,Camio...",Tucson - Ingresa tus datos y te contactaremos ...,"Hyundai,TUCSON,2018,5,4x2,Plata","Parlantes/Bajos,Retrovisores Eléctricos,Radio ...",https://neoauto.com/auto/usado/hyundai-tucson-...
7,Hyundai New Tucson 2018,"US$ 24,900","La Victoria, Lima, Lima","2018,29,915 km,Automática - Secuencial,Dual,2,...",*** HYUNDAI NEW TUCSON *** VERSIÓN GL / AÑO. 2...,"Hyundai,TUCSON,2017,5,4x2,Azul,4","Computadora de abordo,Retrovisores Eléctricos,...",https://neoauto.com/auto/usado/hyundai-new-tuc...
8,Hyundai Hd 78 2017,"US$ 34,900","La Victoria, Lima, Lima","2017,21,733 km,Mecánica,Diesel,3,900 cc,Impar,...",- HYUNDAI - Versión: HD-78 TDI CRDI EURO III -...,"Hyundai,HD 78,2017,2,Posterior,Azul,4,2,4,16,M...","Radio CD,Neblineros,Radio MP3,Aire acondiciona...",https://neoauto.com/bus-camion/usado/hyundai-h...
9,Hyundai Santa Fe 2019,"US$ 35,900","La Victoria, Lima, Lima","2019,78,087 km,Automática - Secuencial,Gasolin...",- HYUNDAI - Versión: SANTA FE GLS - AÑO. 2019 ...,"Hyundai,SANTA FE,2019,4,4x4,4","Computadora de abordo,Retrovisores Eléctricos,...",https://neoauto.com/auto/usado/hyundai-santa-f...


In [34]:
# Hemos obtenido información de 6,100 vehículos.
df.shape

(6100, 8)

In [35]:
# Descargamos la información en excel
df.to_excel('df.xlsx')

In [36]:
# Descargamos la información en csv
df.to_csv('df.csv')

### Próximos pasos

En un siguiente proyecto efectuaremos una limpieza del dataset para obtener cada uno de las caracaterísticas de los vehīculos y sea posible su análisis. Adicional a ello, crear un modelo que nos permita predecir el precio de mercado en función a las características de cualquier vehículo.